In [1]:
import sourcecode.Text.generate
import $repo.`https://repos.spark-packages.org/`
import $ivy.`LLNL:spark-hdf5:0.0.4`
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SQLContext
import gov.llnl.spark.hdf._

import sourcecode.Text.generate
import $repo.$
import $ivy.$
import $ivy.$
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SQLContext
import gov.llnl.spark.hdf._

In [2]:
val spark = SparkSession.builder.master("spark://spark:7077").getOrCreate()
val sqlContext = spark.sqlContext

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/12/06 15:09:08 INFO SparkContext: Running Spark version 3.5.0
24/12/06 15:09:08 INFO SparkContext: OS info Linux, 6.8.0-49-generic, amd64
24/12/06 15:09:08 INFO SparkContext: Java version 1.8.0_402
24/12/06 15:09:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/06 15:09:08 INFO ResourceUtils: ==============================================================
24/12/06 15:09:08 INFO ResourceUtils: No custom resources configured for spark.driver.
24/12/06 15:09:08 INFO ResourceUtils: ==============================================================
24/12/06 15:09:08 INFO SparkContext: Submitted application: 577d6f61-d562-4400-bb0a-c5be486c0737
24/12/06 15:09:08 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 

spark: SparkSession = org.apache.spark.sql.SparkSession@2a48a5b3
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@5f3938d7

In [3]:
val file = "ID16994/Day4/DataFrame_Imaging_spiking_16994_day4.h5"

file: String = "ID16994/Day4/DataFrame_Imaging_spiking_16994_day4.h5"

In [ ]:
val reader = spark.read.format("org.hdfgroup.spark.hdf5").option("path",s"/home/jovyan/data/$file")

24/12/06 15:09:11 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/12/06 15:09:11 INFO SharedState: Warehouse path is 'file:/home/jovyan/spark-warehouse'.


reader: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@5db1435b

In [4]:
val df = sqlContext.read.hdf5(s"/home/jovyan/data/$file", "/df/block0_items")
df.show

24/12/06 15:04:05 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/12/06 15:04:05 INFO SharedState: Warehouse path is 'file:/home/jovyan/spark-warehouse'.


java.lang.NoSuchMethodError: gov.llnl.spark.hdf.package$HDF5DataFrameReader.hdf5(Ljava/lang/String;Ljava/lang/String;)Lorg/apache/spark/sql/Dataset;

In [ ]:
"""
Loads h5 files into the main database.
"""

#%%
import pandas as pd
from glob import glob
import os
import h5py
import yaml
from dotenv import load_dotenv
from tqdm import tqdm
import logging
from pyspark.sql import SparkSession

load_dotenv()

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.propagate = False

INGEST_PATH = os.environ.get("INGEST_PATH")
SPARK_MASTER_URL = os.environ.get("SPARK_MASTER_URL")
TARGET_PATH = os.environ.get("TARGET_PATH")
#%%
#Prefereably run at least the master locally
spark = SparkSession.builder.master(SPARK_MASTER_URL).getOrCreate()

24/12/06 10:09:16 WARN Utils: Your hostname, aigo resolves to a loopback address: 127.0.1.1; using 131.220.127.56 instead (on interface eno1)
24/12/06 10:09:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 10:09:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/12/06 10:09:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
def load_hdf(file, entry_point):
    df = pd.read_hdf(file) #Should automatically get the only key in the file
    df['file_name'] = file[len(entry_point)+1:]
    
    #Join neural data into an array data type (dimensionality may vary)
    #They are the numerical columns, so filter by that.
    numerical_columns = list(filter(lambda x: isinstance(x, int), df.columns))
    
    df["neural_data"] = df[numerical_columns].values.copy().tolist()
    df = df.drop(columns = numerical_columns)
    
    return df

24/12/06 09:50:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
spark.read.format("hdf").load(file).show()

Py4JJavaError: An error occurred while calling o30.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: hdf. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:52)
	at java.